**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Nov 12 06:11:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15353, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15353 (delta 54), reused 53 (delta 53), pack-reused 15293
Receiving objects: 100% (15353/15353), 13.98 MiB | 26.46 MiB/s, done.
Resolving deltas: 100% (10323/10323), done.


**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
# !cp /content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [7]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-11-12 06:12:56--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   104MB/s    in 1.5s    

2021-11-12 06:12:57 (104 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/obj/S2-O1301M01092.jpg  
  inflating: data/obj/S2-O1301M01092.txt  
  inflating: data/obj/S2-O1301M01093.jpg  
  inflating: data/obj/S2-O1301M01093.txt  
  inflating: data/obj/S2-O1301M01094.jpg  
  inflating: data/obj/S2-O1301M01094.txt  
  inflating: data/obj/S2-O1301M01095.jpg  
  inflating: data/obj/S2-O1301M01095.txt  
  inflating: data/obj/S2-O1301M01096.jpg  
 extracting: data/obj/S2-O1301M01096.txt  
  inflating: data/obj/S2-O1301M01097.jpg  
 extracting: data/obj/S2-O1301M01097.txt  
  inflating: data/obj/S2-O1301M01098.jpg  
 extracting: data/obj/S2-O1301M01098.txt  
  inflating: data/obj/S2-O1301M01099.jpg  
 extracting: data/obj/S2-O1301M01099.txt  
  inflating: data/obj/S2-O1301M01103.jpg  
  inflating: data/obj/S2-O1301M01103.txt  
  inflating: data/obj/S2-O1301M01104.jpg  
 extracting: data/obj/S2-O1301M01104.txt  
  inflating: data/obj/S2-O1301M01108.jpg  
  inflating: data/obj/S2-O1301M01108.txt  
  inflating: data/

In [22]:
import glob
images_list = glob.glob("data/obj/*.jpg")
#images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/S2-O1301M04461.jpg', 'data/obj/S2-O1301M02087.jpg', 'data/obj/S2-O1301M01156.jpg', 'data/obj/S2-O1301M01185.jpg', 'data/obj/S2-O1301M00011.jpg', 'data/obj/S2-O1301M03847.jpg', 'data/obj/S2-O1301M03704.jpg', 'data/obj/S2-O1301M00780.jpg', 'data/obj/S2-O1301M00169.jpg', 'data/obj/S2-O1301M02900.jpg', 'data/obj/S2-O1301M00057.jpg', 'data/obj/S2-O1301M03863.jpg', 'data/obj/S2-O1301M01750.jpg', 'data/obj/S2-O1301M03053.jpg', 'data/obj/S2-O1301M03061.jpg', 'data/obj/S2-O1301M04295.jpg', 'data/obj/S2-O1301M02082.jpg', 'data/obj/S2-O1301M01043.jpg', 'data/obj/S2-O1302M00404.jpg', 'data/obj/S2-O1302M00068.jpg', 'data/obj/S2-O1301M03625.jpg', 'data/obj/S2-O1301M04012.jpg', 'data/obj/S2-O1301M03969.jpg', 'data/obj/S2-O1301M00682.jpg', 'data/obj/S2-O1301M02023.jpg', 'data/obj/S2-O1301M01090.jpg', 'data/obj/S2-O1302M00570.jpg', 'data/obj/S2-O1301M00657.jpg', 'data/obj/S2-O1301M04046.jpg', 'data/obj/S2-O1301M03690.jpg', 'data/obj/S2-O1301M00650.jpg', 'data/obj/S2-O1301M04304.jpg', 'data/o

In [24]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**5) Start the training**

In [25]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.706959), count: 5, class_loss = 0.385451, iou_loss = 0.188642, total_loss = 0.574094 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000180, iou_loss = 0.000000, total_loss = 0.000180 
 total_bbox = 124033, rewritten_bbox = 3.155612 % 

 5535: 1.127282, 1.052646 avg loss, 0.000010 rate, 0.231899 seconds, 44280 images, 0.069551 hours left
Loaded: 0.000053 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.808315), count: 19, class_loss = 1.255028, iou_loss = 0.466576, total_loss = 1.721604 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.731974), count: 4, class_loss = 0.283331, iou_loss = 0.159063, total_loss = 0.442394 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_lo

In [28]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_final.weights -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 8, batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 